In [46]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import cartopy.crs as ccrs
from pyresample import geometry, grid
import pyresample
import matplotlib.pyplot as plt
import datetime
import metpy
import datetime
import xarray as xr
from pyresample import geometry, grid
import cartopy.crs as ccrs
from sklearn.metrics import mean_squared_error
import math
from skimage.transform import resize
from skimage.transform import resize_local_mean
import tensorflow as tf
import tensorflow_hub as hub
from skimage.filters import laplace
from skimage.filters import unsharp_mask

model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

from sharpening_functions import *
from cropping_functions import *

In [125]:
pc = ccrs.PlateCarree()
pc_params = pc.proj4_params
shape = [500, 500]
pc_extents = [-124.25, 32.25, -114.25, 42.25]
target_area = geometry.AreaDefinition.from_extent('CA', pc_params, shape, pc_extents)

goes_path = '../landsat_files/'
landsat_path = '../landsat_files/landsat_files/'

day1 = {'goes':'2021240', 'landsat':'20210828'}
day2 = {'goes':'2021247', 'landsat':'20210904'}

a1 = {'day':day1, 'loc':'A', 'lat_0':40.49, 'lon_0':-121.45, 'lat_f':40.650000000000006, 'lon_f':-121.17}
b1 = {'day':day1, 'loc':'B', 'lat_0':39.91, 'lon_0':-120.97, 'lat_f':40.13, 'lon_f':-120.49000000000001}
a2 = {'day':day2, 'loc':'A', 'lat_0':40.707499999999996, 'lon_0':-123.4325, 'lat_f':41.1325, 'lon_f':-122.7075}
b2 = {'day':day2, 'loc':'B', 'lat_0':41.027499999999996, 'lon_0':-123.0925, 'lat_f':41.31250000000001, 'lon_f':-122.6875}

subsets = [a1, b1, a2, b2]

In [126]:
subset = a1

for file in os.listdir(landsat_path):
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT3.9_grid' + subset['loc'] + 'cent0.02deg.npy':
        landsat_img_039 = np.flipud(np.load(landsat_path + file))
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT10.3_grid' + subset['loc'] + 'cent0.02deg.npy':
        landsat_img_103 = np.flipud(np.load(landsat_path + file))
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT3.9_grid' + subset['loc'] + 'cent0.01deg.npy':
        new_landsat_img_039 = np.flipud(np.load(landsat_path + file))
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT10.3_grid' + subset['loc'] + 'cent0.01deg.npy':
        new_landsat_img_103 = np.flipud(np.load(landsat_path + file))

for file in os.listdir(goes_path):
    if file[:34] == 'OR_ABI-L1b-RadC-M6C07_G17_s' + subset['day']['goes']:
        goes_img_039 = np.load(goes_path + file)
    if file[:34] == 'OR_ABI-L1b-RadC-M6C14_G17_s' + subset['day']['goes']:
        goes_img_103 = np.load(goes_path + file)

new_landsat_diff = new_landsat_img_039 - new_landsat_img_103
subset['landsat_diff'] = new_landsat_diff
subset['landsat_img_039'] = new_landsat_img_039
subset['landsat_img_103'] = new_landsat_img_103

In [127]:
latslons(subset)
pixshifts(subset)

In [128]:
for file in os.listdir(landsat_path):
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT3.9_grid' + subset['loc'] + 'cent0.02deg.npy':
        landsat_img_039 = np.flipud(np.load(landsat_path + file))
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT10.3_grid' + subset['loc'] + 'cent0.02deg.npy':
        landsat_img_103 = np.flipud(np.load(landsat_path + file))
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT3.9_grid' + subset['loc'] + 'cent0.01deg.npy':
        new_landsat_img_039 = np.flipud(np.load(landsat_path + file))
    if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT10.3_grid' + subset['loc'] + 'cent0.01deg.npy':
        new_landsat_img_103 = np.flipud(np.load(landsat_path + file))

for file in os.listdir(goes_path):
    if file[:34] == 'OR_ABI-L1b-RadC-M6C07_G17_s' + subset['day']['goes']:
        goes_img_039 = np.load(goes_path + file)
    if file[:34] == 'OR_ABI-L1b-RadC-M6C14_G17_s' + subset['day']['goes']:
        goes_img_103 = np.load(goes_path + file)

new_landsat_diff = new_landsat_img_039 - new_landsat_img_103
subset['landsat_diff'] = new_landsat_diff
subset['landsat_img_039'] = new_landsat_img_039
subset['landsat_img_103'] = new_landsat_img_103

goes_img_expanded_039 = expand(landsat_img_039, goes_img_039, subset['shift'][0], subset['shift'][1], subset['lats'], subset['lons'])
goes_img_expanded_103 = expand(landsat_img_103, goes_img_103, subset['shift'][0], subset['shift'][1], subset['lats'], subset['lons'])

In [129]:
goes_img_expanded_039.shape

(12, 18)

In [130]:
def shiftPixels_old(refimg, newimg_full, lats=None, lons=None):
    assert refimg.shape[0] <= newimg_full.shape[0]
    assert refimg.shape[1] <= newimg_full.shape[1]
    
    if lats != None or lons != None:
        newimg = newimg_full[lats[0]:lats[1], lons[0]:lons[1]]
    else:
        newimg = newimg_full.copy()
    
    nx = 0
    ny = 0
    
    x_add = 0
    y_add = 0
        
    while refimg.shape != newimg.shape:
        if refimg.shape[0] > newimg.shape[0]:
            x_add += 1
            nx += 1
        elif refimg.shape[0] < newimg.shape[0]:
            x_add -= 1
            nx += 1
        if refimg.shape[1] > newimg.shape[1]:
            y_add += 1
            ny += 1
        elif refimg.shape[1] < newimg.shape[1]:
            y_add -= 1
            ny += 1
        
        if lats != None or lons != None:
            newimg = newimg_full[lats[0]:lats[1] + nx, lons[0]:lons[1] + ny]
        else:
            newimg = newimg_full[:(newimg.shape[0] + x_add), :(newimg.shape[1] + y_add)]
    
    if lats != None or lons != None:
        lats[1] += x_add
        lons[1] += y_add

    nx = 1
    ny = 1
    x_shift = np.arange(-nx, nx+1)
    y_shift = np.arange(-ny, ny+1)
    scores = []
    
    for x in x_shift:
        for y in y_shift:
            if lats != None or lons != None:
                testimg = newimg_full[lats[0]+x:lats[1]+x+1, lons[0]+y:lons[1]+y+1]
            else:
                testimg = newimg_full[abs(x):(newimg.shape[0]+abs(x)), abs(y):(newimg.shape[1]+abs(y))]
            scores.append([x, y, np.corrcoef(refimg.flatten(), testimg.flatten())[0, 1]])
          
    max_score = 0
    min_score_index = 0
    
    for score_index in range(len(scores)-1):
        score = scores[score_index][-1]
        if abs(score) > abs(max_score):
            max_score = score
            max_score_index = score_index
            
    min_x, min_y, max_score = scores[max_score_index]
    print(min_x, min_y)
    if lats != None or lons != None:
        return newimg_full[lats[0]+min_x:lats[1]+min_x, lons[0]+min_y:lons[1]+min_y]
    return newimg_full[abs(min_x):(newimg.shape[0] + abs(min_x)), abs(min_y):(newimg.shape[1] + abs(min_y))]

In [131]:
def control_img(img):
    from skimage.transform import resize
    height, width = img.shape
    control = resize(img, (2*height, 2*width))
    return control

In [154]:
subset=a1

## 3.9 and 10.3 have different shifts, one the same as diff and one different

In [147]:
a1['shift']

[-1, 0]

In [155]:
control1_goes_img_039 = shiftPixels_old(new_landsat_img_039, control_img(goes_img_expanded_039))
control1_goes_img_103 = shiftPixels_old(new_landsat_img_103, control_img(goes_img_expanded_103))

0 0
0 -1


In [157]:
control_goes_img_039 = shiftPixels(new_landsat_img_039, control_img(goes_img_expanded_039), subset['shift'][0], subset['shift'][1])
control_goes_img_103 = shiftPixels(new_landsat_img_103, control_img(goes_img_expanded_103), subset['shift'][0], subset['shift'][1])

In [163]:
control2_goes_img_039 = shiftPixels(new_landsat_img_039, control_img(goes_img_expanded_039), 0, 0)
control2_goes_img_103 = shiftPixels(new_landsat_img_103, control_img(goes_img_expanded_103), 0, 0)

In [158]:
control_goes_diff = control_goes_img_039 - control_goes_img_103
control_goes_img_039 = control_goes_img_039
control_goes_img_039[control_goes_img_039 > 373] = 373
control_goes_img_103 = control_goes_img_103

In [165]:
control2_goes_diff = control2_goes_img_039 - control2_goes_img_103
control2_goes_img_039 = control2_goes_img_039
control2_goes_img_039[control2_goes_img_039 > 373] = 373
control2_goes_img_103 = control2_goes_img_103

In [160]:
def results(refimg, newimg, title):
    print(title + " correlation coefficient: " + str(np.corrcoef(refimg.ravel(), newimg.ravel())[0, 1]))
    print(title + " rmse: " + str(math.sqrt(np.square(refimg - newimg).mean())))
    return [abs(np.corrcoef(refimg, newimg).mean()), math.sqrt(np.square(refimg - newimg).mean())/50]

In [161]:
results(subset['landsat_diff'], control_goes_diff, 'Control')

Control correlation coefficient: 0.27918191477907506
Control rmse: 7.286482451154725


[0.16816526082856706, 0.1457296490230945]

In [166]:
results(subset['landsat_diff'], control2_goes_diff, 'Control')

Control correlation coefficient: 0.28750096294123967
Control rmse: 7.214067241979874


[0.17809986745898437, 0.14428134483959748]

In [ ]:
print('max bt 373 stats \n')

titles = ['ESRGAN', 'Laplace', 'Unsharp Mask', 'Control']

latslons(subset)
pixshifts(subset)
applySR(subset)

corrcoeffs = []
rmses = []

print('Subset ' + subset['loc'])
stats = [subset['esrgan_goes_diff'], subset['laplace_goes_diff'], subset['unsharpmask_goes_diff'], subset['control_goes_diff']]
for n in range(len(titles)):
    vals = results(subset['landsat_diff'], stats[n], titles[n])
    corrcoeffs.append(vals[0])
    rmses.append(vals[1])

subset['corrcoeffs'] = corrcoeffs
subset['rmses'] = rmses

print('')

In [18]:
def latslons(subset):
    coords_0 = target_area.get_array_coordinates_from_lonlat(subset['lon_0'], subset['lat_0'])
    coords_0 = (round(coords_0[0]), round(coords_0[1]))
    coords_f = target_area.get_array_coordinates_from_lonlat(subset['lon_f'], subset['lat_f'])
    coords_f = (round(coords_f[0]), round(coords_f[1]))
    subset['lons'] = [coords_0[0], coords_f[0]]
    subset['lats'] = [coords_f[1], coords_0[1]]

def pixshifts(subset):
    for file in os.listdir(landsat_path):
        if file[17:25] == subset['day']['landsat'] and file[48:] == 'red_grid' + subset['loc'] + 'cent0.02deg.npy':
            refimg = np.flipud(np.load(landsat_path + file))

    for file in os.listdir(goes_path):
        if file[:34] == 'OR_ABI-L1b-RadC-M6C02_G17_s' + subset['day']['goes']:
            newimg_full = np.load(goes_path + file)
    
    lats = subset['lats']
    lons = subset['lons']

    newimg = newimg_full[lats[0]:lats[1], lons[0]:lons[1]]
    
    x_add = 0
    y_add = 0
    
    while refimg.shape != newimg.shape: 
        if refimg.shape[0] > newimg.shape[0]:
            x_add += 1
        elif refimg.shape[0] < newimg.shape[0]:
            x_add -= 1
        if refimg.shape[1] > newimg.shape[1]:
            y_add += 1
        elif refimg.shape[1] < newimg.shape[1]:
            y_add -= 1
        
        newimg = newimg_full[lats[0]:lats[1] + x_add, lons[0]:lons[1] + y_add]
    
    lats[1] += x_add
    lons[1] += y_add

    nx = 1
    ny = 1
    x_shift = np.arange(-nx, nx+1)
    y_shift = np.arange(-ny, ny+1)
    scores = []
    
    for x in x_shift:
        for y in y_shift:
            testimg = newimg_full[lats[0]+x:lats[1]+x, lons[0]+y:lons[1]+y]
            scores.append([x, y, np.corrcoef(refimg.flatten(), testimg.flatten())[0, 1]])
    
    max_score = 0
    min_score_index = 0
    
    for score_index in range(len(scores)-1):
        score = scores[score_index][-1]
        if abs(score) > abs(max_score):
            max_score = score
            max_score_index = score_index
            
    min_x, min_y, max_score = scores[max_score_index]
    subset['shift'] = [min_x, min_y]

In [4]:
def applySR(subset):
    for file in os.listdir(landsat_path):
        if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT3.9_grid' + subset['loc'] + 'cent0.02deg.npy':
            landsat_img_039 = np.flipud(np.load(landsat_path + file))
        if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT10.3_grid' + subset['loc'] + 'cent0.02deg.npy':
            landsat_img_103 = np.flipud(np.load(landsat_path + file))
        if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT3.9_grid' + subset['loc'] + 'cent0.01deg.npy':
            new_landsat_img_039 = np.flipud(np.load(landsat_path + file))
        if file[17:25] == subset['day']['landsat'] and file[48:] == 'BT10.3_grid' + subset['loc'] + 'cent0.01deg.npy':
            new_landsat_img_103 = np.flipud(np.load(landsat_path + file))

    for file in os.listdir(goes_path):
        if file[:34] == 'OR_ABI-L1b-RadC-M6C07_G17_s' + subset['day']['goes']:
            goes_img_039 = np.load(goes_path + file)
        if file[:34] == 'OR_ABI-L1b-RadC-M6C14_G17_s' + subset['day']['goes']:
            goes_img_103 = np.load(goes_path + file)
            
    new_landsat_diff = new_landsat_img_039 - new_landsat_img_103
    subset['landsat_diff'] = new_landsat_diff
    subset['landsat_img_039'] = new_landsat_img_039
    subset['landsat_img_103'] = new_landsat_img_103

    goes_img_expanded_039 = expand(landsat_img_039, goes_img_039, subset['shift'][0], subset['shift'][1], subset['lats'], subset['lons'])
    goes_img_expanded_103 = expand(landsat_img_103, goes_img_103, subset['shift'][0], subset['shift'][1], subset['lats'], subset['lons'])

    esrgan_goes_img_039 = shiftPixels(new_landsat_img_039, esrgan_sharpening_img(goes_img_expanded_039), subset['shift'][0], subset['shift'][1])
    esrgan_goes_img_039[esrgan_goes_img_039 > 373] = 373
    esrgan_goes_img_103 = shiftPixels(new_landsat_img_103, esrgan_sharpening_img(goes_img_expanded_103), subset['shift'][0], subset['shift'][1])
    subset['esrgan_goes_diff'] = esrgan_goes_img_039 - esrgan_goes_img_103
    subset['esrgan_goes_img_039'] = esrgan_goes_img_039
    subset['esrgan_goes_img_103'] = esrgan_goes_img_103

    laplace_goes_img_039 = shiftPixels(new_landsat_img_039, laplace_sharpening_img(goes_img_expanded_039), subset['shift'][0], subset['shift'][1])
    laplace_goes_img_039[laplace_goes_img_039 > 373] = 373
    laplace_goes_img_103 = shiftPixels(new_landsat_img_103, laplace_sharpening_img(goes_img_expanded_103), subset['shift'][0], subset['shift'][1])
    subset['laplace_goes_diff'] = laplace_goes_img_039 - laplace_goes_img_103
    subset['laplace_goes_img_039'] = laplace_goes_img_039
    subset['laplace_goes_img_103'] = laplace_goes_img_103

    unsharpmask_goes_img_039 = shiftPixels(new_landsat_img_039, unsharpmask_sharpening_img(goes_img_expanded_039), subset['shift'][0], subset['shift'][1])
    unsharpmask_goes_img_039[unsharpmask_goes_img_039 > 373] = 373
    unsharpmask_goes_img_103 = shiftPixels(new_landsat_img_103, unsharpmask_sharpening_img(goes_img_expanded_103), subset['shift'][0], subset['shift'][1])
    subset['unsharpmask_goes_diff'] = unsharpmask_goes_img_039 - unsharpmask_goes_img_103
    subset['unsharpmask_goes_img_039'] = unsharpmask_goes_img_039
    subset['unsharpmask_goes_img_103'] = unsharpmask_goes_img_103

    control_goes_img_039 = shiftPixels(new_landsat_img_039, control_img(goes_img_expanded_039), subset['shift'][0], subset['shift'][1])
    control_goes_img_103 = shiftPixels(new_landsat_img_103, control_img(goes_img_expanded_103), subset['shift'][0], subset['shift'][1])
    subset['control_goes_diff'] = control_goes_img_039 - control_goes_img_103
    subset['control_goes_img_039'] = control_goes_img_039
    control_goes_img_039[control_goes_img_039 > 373] = 373
    subset['control_goes_img_103'] = control_goes_img_103